In [1]:
deployment = "gpt4"
model = "gpt-4"

# 回顾

In [2]:
import openai
def analyze_user_review(text):
    messages = []
    messages.append( {"role": "system", 
                      "content": """
                      You are an assistant. 
                      Please, analyze the user reviews according to the following instruction: 
                      If the review is postive, you should output 'Y', otherwise output 'N'
                      """})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        engine=deployment, 
        messages=messages,
        temperature=0.5,
        max_tokens = 100
    )
    return response["choices"][0]["message"]["content"]

In [3]:
analyze_user_review("服务热情周到。")

'Y'

In [4]:
analyze_user_review("用餐环境差，等待时间长。")

'N'

In [5]:
!pip install --upgrade langchain

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


# 提示词模版

In [5]:
from langchain import PromptTemplate, OpenAI, LLMChain

from langchain.chat_models import AzureChatOpenAI
# from langchain.chat_models import ChatOpenAI #For Direct GPT client

prompt_template = "What is a good name for a company that makes {product}? And only return the best one."

#llm = OpenAI(temperature=0) #For Direct GPT client
llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain.run("colorful socks")

'Spectrum Socks'

In [8]:
products = [{"product":"'cloudnative devops platform'"},
            {"product":"'Noise cancellation headphone'"}, 
            {"product":"colorful socks"}]
llm_chain.apply(products)

[{'text': 'SkyOps Innovations'},
 {'text': 'Silent Symphony'},
 {'text': 'Spectrum Socks'}]

In [9]:
import time
async for i in llm_chain.astream("colorful socks"):
    print (i)
    time.sleep(1)
    

{'product': 'colorful socks', 'text': 'Spectrum Socks'}


# API调用链
## HTTP request chain

In [6]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMRequestsChain
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

def query_baidu(question):
      template = """Between >>> and <<< are the raw search result text from google.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:"""

      PROMPT = PromptTemplate(
          input_variables=["query", "requests_result"],
          template=template,
      )

      inputs = {
          "query": question,
          "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
      }
      requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info")
      res = requests_chain.run(inputs)
      return res

In [7]:
query_baidu("今天北京天气？")

'10:50更新25°多云东北风2级21~30°C38 优'

In [1]:
import os

# os.environ["BING_SUBSCRIPTION_KEY"] = "<Your Key>"
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

from langchain.utilities import BingSearchAPIWrapper

def query_web(question):
    search = BingSearchAPIWrapper()
    return search.run(question)


In [2]:
query_web("今天北京天气？")

'<b>今天</b> 7天 8-15天 40天 雷达图 31日（<b>今天</b>） 晴 29 / 19℃ &lt;3级 1日（明天） 多云 30 / 21℃ &lt;3级 2日（后天） 小雨 28 / 20℃ &lt;3级 3日（周日） 小雨 28 / 19℃ &lt;3级 4日（周一） 多云 29 / 20℃ &lt;3级 5日（周二） 晴 30 / 21℃ &lt;3级 6日（周三） 晴 31 / 21℃ &lt;3级 分时段预报 生活指数 蓝天预报 <b>天气</b>资讯 全国开学<b>天气</b>地图出炉 北方大部秋高气爽广东福建开学“撞”台风 中国<b>天气</b>网 2023-08-31 11:20 湖南大部<b>今天</b>阳光在线 明晚起南部将有较强降雨过程局地暴雨 中国<b>天气</b>网湖南站 2023-08-31 11:10 省份： 城市：. 日出05:39. 北京. 日落18:51. 降水量. 相对湿度. 体感温度. 雷达图. 发布时间：08-30 12:00. 大部晴朗. Feels-Like Temperature. 体感温度 20°. Wind. 大风北 7 公里/小时. Humidity. 湿度 68%. UV Level. 紫外线指数 0（最大值11）. 【<b>北京天气</b>】<b>北京天气</b>预报,<b>天气</b>预报一周,<b>天气</b>预报15天查询 <b>北京</b> &gt; 城区 <b>今天</b> 7天 8-15天 更长时间 雷达图 29日（<b>今天</b>） 霾转晴 4 / -4℃ 微风 30日（明天） 晴 5 / -6℃ 3-4级转微风 31日（后天） 晴转多云 4 / -6℃ 微风 1日（周五） 多云转晴 4 / -4℃ 微风 2日（周六） 晴转多云 5 / -4℃ 3-4级转微风 3日（周日） 多云转晴 4 / -4℃ 微风 4日（周一） 晴转多云 4 / -2℃ 微风 分时段预报 生活指数 蓝天预报 周边地区 | 周边景点 2023-08-31 18:00更新 香河 / 7天天气预报（2023/08/30 08:00发布）. 星期三. 08/30. 多云. 无持续风向. 微风. 29℃. 19℃. 多云. <b>北京天气</b>网为您提供<b>北京天气</b>预报24小时详情、<b>北京</b

# 调用链

In [8]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain

llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

summarizing_prompt_template = """
Summarize the following content into a sentence less than 20 words:
---
{content}

"""
summarizing_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(summarizing_prompt_template),
    output_key = "summary"
)

translating_prompt_template = """
translate "{summary}" into Chinese:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)

overall_chain = SequentialChain(
    chains=[summarizing_chain, translating_chain],
    input_variables=["content"],
    output_variables=[ "summary","translated"],
    verbose=True
)


In [9]:
res = overall_chain("""
LangChain is a framework for developing applications powered by language models. It enables applications that are:

Data-aware: connect a language model to other sources of data
Agentic: allow a language model to interact with its environment
The main value props of LangChain are:

Components: abstractions for working with language models, along with a collection of implementations for each abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not
Off-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks
Off-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it easy to customize existing chains or build new ones.
""")

print("summary:"+res["summary"])

print("中文:"+res["translated"])



> Entering new SequentialChain chain...

> Finished chain.
summary:LangChain is a framework for creating applications powered by language models, offering data-aware and agentic features, modular components, and off-the-shelf chains for easy customization.
中文:"LangChain是一个用于创建由语言模型驱动的应用程序的框架，提供数据感知和代理特性，模块化组件，以及现成的链条以便于定制。"
